In [1]:
import geopandas as gpd
import pandas as pd

pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

# cities = gpd.read_feather(
#     "/Users/test/Documents/code/IL2/factory_loc_service/examples/closest_cities.feather"
# )
cities = gpd.read_parquet(
    "/Users/test/Documents/code/IL2/factory_loc_service/api/app/data/cities.parquet"
)

uinput_industry = "oil_and_gas_ext"
plant_attr_coef = 0.1
uinput_spec_num = {"Машинист": 200, "Оператор, аппаратчик": 300}
# city_names = cities["region_city"].values
city_names = [
    "Ростовская область, Азов",
    "Ростовская область, Аксай",
    "Ростовская область, Батайск",
    "Ростовская область, Белая Калитва",
    "Ростовская область, Зверево",
    "Ростовская область, Каменск-Шахтинский",
    "Ростовская область, Новочеркасск",
    "Ростовская область, Ростов-на-Дону",
    "Ростовская область, Таганрог",
    "Краснодарский край, Тихорецк",
    "Ростовская область, Шахты",
]

In [2]:
# print(uinput_industry, uinput_spec_num, city_names)

In [3]:
ontology = pd.read_pickle(
    "/Users/test/Documents/code/IL2/factory_loc_service/api/app/data/new_ontology.pkl"
)

grouped_grads = pd.read_pickle(
    "/Users/test/Documents/code/IL2/factory_loc_service/api/app/data/grouped_grads.pkl"
)

cv = pd.read_parquet("../api/app/data/cv.gzip").rename(
    columns={"hh_name": "speciality"}
)
YEAR = 2021
cv = cv[cv["year"] == YEAR]

In [4]:
mask = grouped_grads.reset_index(drop=False).loc[:, "cluster_center"].isin(city_names)
grouped_grads = (
    grouped_grads.reset_index(drop=False)
    .loc[mask, :]
    .set_index(["cluster_center", "region_city", "type", "edu_group_code"])
)

mask = cv["region_city"].isin(city_names)
cv = cv[mask]

In [5]:
# ontology = pd.read_csv(
#     "/Users/test/Documents/code/IL2/industrial-location/api/app/data/ontology.csv",
#     index_col=0,
# )

# ontology.loc[ontology["industry_code"] == "mining_oil_gas", "industry_code"] = (
#     "oil_and_gas_ext"
# )
# ontology.loc[ontology["industry_code"] == "mmining_coal_ores", "industry_code"] = (
#     "coal_and_metal_ext"
# )
# ontology.loc[ontology["industry_code"] == "mech_engineering", "industry_code"] = (
#     "machinery"
# )
# ontology.loc[ontology["industry_code"] == "shipbuilding", "industry_code"] = (
#     "shipbuilding"
# )
# ontology.loc[ontology["industry_code"] == "aircraft_engineering", "industry_code"] = (
#     "aircraft_and_space"
# )
# ontology.loc[ontology["industry_code"] == "non_ferrous_metallurgy", "industry_code"] = (
#     "nonferrous_metallurgy"
# )
# ontology.loc[ontology["industry_code"] == "ferrous_metallurgy", "industry_code"] = (
#     "ferrous_metallurgy"
# )
# ontology.loc[ontology["industry_code"] == "chemical", "industry_code"] = "chemicals"
# ontology.loc[ontology["industry_code"] == "pharma", "industry_code"] = "pharmacy"
# ontology.loc[ontology["industry_code"] == "electronics", "industry_code"] = (
#     "electronics"
# )

In [6]:
# ontology["industry_code"].unique()

In [7]:
# ontology.to_pickle("new_ontology.pkl")

In [8]:
cv = cv.merge(ontology[["speciality", "edu_group_code"]], on="speciality")
cv["type"] = "CV"
grouped_cv = (
    cv.groupby(["cluster_center", "region_city", "type", "edu_group_code"])["id_cv"]
    .count()
    .to_frame()
)
cv.head(3)

,id_cv,speciality,year,region_city,cluster_center,edu_group_code,type
0,48f13960-0043-11ec-884c-6db06c9eaf56,"Оператор, аппаратчик",2021,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",21,CV
1,48f13960-0043-11ec-884c-6db06c9eaf56,"Оператор, аппаратчик",2021,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",21,CV
2,48f13960-0043-11ec-884c-6db06c9eaf56,"Оператор, аппаратчик",2021,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",15,CV


In [9]:
# grads = pd.read_csv("graduates.csv")
# grads.dropna(subset="edu_group", inplace=True)
# grouped_grads = (
#     grads.groupby(["cluster_center", "region_city", "type", "edu_group_code"])[
#         "graduates_per_year_forecast"
#     ]
#     .sum()
#     .to_frame()
# )
# grouped_grads = grouped_grads.join(grouped_cv, how="outer")
# grouped_grads["city_capacity_grads_and_cv_sum"] = grouped_grads[
#     "graduates_per_year_forecast"
# ].fillna(0) + grouped_grads["id_cv"].fillna(0)
# grouped_grads.to_pickle("grouped_grads.pkl")

In [10]:
# --- START ---
"""
Эта штука завязана на входных профессиях и индустрии,
поэтому нужно каждый раз пересчитывать -- мб можно оптимизировать и пересчитывать только часть
но имхо оно и так быстро
"""

uinput_spec_num_2 = dict()
competitor_industries = list()
competitor_fatories = list()


for k in uinput_spec_num.keys():
    uinput_spec_num_2[k] = (
        ontology.loc[ontology["speciality"] == k, "edu_group_id"]
        .drop_duplicates()
        .values.tolist()
    )
    competitor_industries += (
        ontology.loc[ontology["speciality"] == k, "industry_code"]
        .drop_duplicates()
        .tolist()
    )
# --- START ---

competitor_industries = set(competitor_industries)
competitor_industries.remove(uinput_industry)


grad_col = []
fact_col = []

for col in cities.columns:
    if "factor" in col and uinput_industry in col:
        fact_col.append(col)

    if "factor" in col and any(industry in col for industry in competitor_industries):
        competitor_fatories.append(col)

    if "grad" in col and uinput_industry in col:
        grad_col.append(col)

cities["competitors_factories_num"] = cities[competitor_fatories].sum(axis=1)

print(competitor_industries)
print(grad_col, fact_col, competitor_fatories)

{'machinery', 'mining_coal_ores', 'shipbuilding', 'ferrous_metallurgy', 'electronics', 'nonferrous_metallurgy', 'pharmacy', 'chemicals'}
[] [] []


In [11]:
uinput_spec_num_2

{'Машинист': [0, 11, 12],
 'Оператор, аппаратчик': [0, 3, 6, 11, 12, 15, 16, 20]}

In [12]:
groups = []

for k, v in uinput_spec_num_2.items():
    groups += v

In [13]:
groups = set(groups)

In [14]:
groups

{0, 3, 6, 11, 12, 15, 16, 20}

In [15]:
cv.drop_duplicates(subset=["speciality", "edu_group_code"])

,id_cv,speciality,year,region_city,cluster_center,edu_group_code,type
0,48f13960-0043-11ec-884c-6db06c9eaf56,"Оператор, аппаратчик",2021,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",21,CV
2,48f13960-0043-11ec-884c-6db06c9eaf56,"Оператор, аппаратчик",2021,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",15,CV
3,48f13960-0043-11ec-884c-6db06c9eaf56,"Оператор, аппаратчик",2021,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",26,CV
4,48f13960-0043-11ec-884c-6db06c9eaf56,"Оператор, аппаратчик",2021,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",22,CV
5,48f13960-0043-11ec-884c-6db06c9eaf56,"Оператор, аппаратчик",2021,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",18,CV
9,48f13960-0043-11ec-884c-6db06c9eaf56,"Оператор, аппаратчик",2021,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",33,CV
11,48f13960-0043-11ec-884c-6db06c9eaf56,"Оператор, аппаратчик",2021,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",19,CV
12,48f13960-0043-11ec-884c-6db06c9eaf56,"Оператор, аппаратчик",2021,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",11,CV
14,365ff700-66d8-11eb-822c-736ab11edb0c,Инженер-проектировщик,2021,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",15,CV
15,365ff700-66d8-11eb-822c-736ab11edb0c,Инженер-проектировщик,2021,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",26,CV


In [16]:
grouped_grads

speciality  graduates_per_year_forecast  cv_count  city_capacity_grads_and_cv_sum
cluster_center               region_city                       type edu_group_code                                                                                     
Краснодарский край, Тихорецк Краснодарский край, Архангельская CV   8               Разнорабочий                          NaN       1.0                             1.0
                                                                    11              Разнорабочий                          NaN       1.0                             1.0
                                                                    15              Разнорабочий                          NaN       2.0                             2.0
                                                                    15                   Слесарь                          NaN       5.0                             5.0
                                                                    21              Разнорабочий                          NaN       2.0                             2.0
...                                                                                          ...                          ...       ...                             ...
Ростовская область, Шахты    Ростовская область, Шахты         СПО  11                       NaN                         43.0       NaN                            43.0
                                                                    15                       NaN                        160.0       NaN                           160.0
                                                                    18                       NaN                        109.0       NaN                           109.0
                                                                    19                       NaN                         77.0       NaN                            77.0
                                                                    21                       NaN                         57.0       NaN                            57.0

[1699 rows x 4 columns]

In [17]:
grouped_grads.reset_index(drop=False, inplace=True)
mask_groups = grouped_grads["edu_group_code"].isin(groups)
grouped_grads = grouped_grads[mask_groups]

In [18]:
grouped_grads

,cluster_center,region_city,type,edu_group_code,speciality,graduates_per_year_forecast,cv_count,city_capacity_grads_and_cv_sum
1,"Краснодарский край, Тихорецк","Краснодарский край, Архангельская",CV,11,Разнорабочий,NaN,1.0,1.0
2,"Краснодарский край, Тихорецк","Краснодарский край, Архангельская",CV,15,Разнорабочий,NaN,2.0,2.0
3,"Краснодарский край, Тихорецк","Краснодарский край, Архангельская",CV,15,Слесарь,NaN,5.0,5.0
9,"Краснодарский край, Тихорецк","Краснодарский край, Атаманская",CV,11,Разнорабочий,NaN,1.0,1.0
10,"Краснодарский край, Тихорецк","Краснодарский край, Атаманская",CV,15,Разнорабочий,NaN,2.0,2.0
...,...,...,...,...,...,...,...,...
1652,"Ростовская область, Шахты","Ростовская область, Шахты",CV,15,"Токарь, фрезеровщик, шлифовщик",NaN,25.0,25.0
1688,"Ростовская область, Шахты","Ростовская область, Шахты",ВПО,11,NaN,348.0,NaN,348.0
1689,"Ростовская область, Шахты","Ростовская область, Шахты",ВПО,15,NaN,512.0,NaN,512.0
1694,"Ростовская область, Шахты","Ростовская область, Шахты",СПО,11,NaN,43.0,NaN,43.0


In [19]:
grouped_grads.loc[grouped_grads["type"] == "ВПО"]

,cluster_center,region_city,type,edu_group_code,speciality,graduates_per_year_forecast,cv_count,city_capacity_grads_and_cv_sum
476,"Ростовская область, Азов","Ростовская область, Азов",ВПО,15,NaN,145.0,NaN,145.0
1036,"Ростовская область, Каменск-Шахтинский","Ростовская область, Каменск-Шахтинский",ВПО,15,NaN,171.0,NaN,171.0
1226,"Ростовская область, Новочеркасск","Ростовская область, Новочеркасск",ВПО,11,NaN,765.0,NaN,765.0
1227,"Ростовская область, Новочеркасск","Ростовская область, Новочеркасск",ВПО,15,NaN,1807.0,NaN,1807.0
1324,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",ВПО,6,NaN,1224.0,NaN,1224.0
1325,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",ВПО,11,NaN,3694.0,NaN,3694.0
1326,"Ростовская область, Ростов-на-Дону","Ростовская область, Ростов-на-Дону",ВПО,15,NaN,1265.0,NaN,1265.0
1562,"Ростовская область, Таганрог","Ростовская область, Таганрог",ВПО,15,NaN,321.0,NaN,321.0
1688,"Ростовская область, Шахты","Ростовская область, Шахты",ВПО,11,NaN,348.0,NaN,348.0
1689,"Ростовская область, Шахты","Ростовская область, Шахты",ВПО,15,NaN,512.0,NaN,512.0


In [20]:
grouped_grads.loc[grouped_grads["type"] == "ВПО", "type"] = "СПО"
grouped_grads.loc[grouped_grads["type"] == "СПО", "type"] = "graduates"
grouped_grads.loc[grouped_grads["type"] == "CV", "type"] = "specialists"

grouped_grads = (
    grouped_grads.set_index("cluster_center")
    .groupby(["cluster_center", "type", "edu_group_code"])[
        ["city_capacity_grads_and_cv_sum", "graduates_per_year_forecast", "cv_count"]
    ]
    .sum()
    .join(
        cities[
            ["region_city", "population", "num_in_migration"]
            + grad_col
            + fact_col
            + ["factories_total"]
            + ["competitors_factories_num"]
        ]
        .rename(columns={"region_city": "cluster_center"})
        .set_index("cluster_center"),
        how="left",
    )
)

grouped_grads

city_capacity_grads_and_cv_sum  graduates_per_year_forecast  cv_count  population  num_in_migration  factories_total  competitors_factories_num
cluster_center                         type        edu_group_code                                                                                                                                                 
Краснодарский край, Тихорецк           graduates   11                                        88.0                         88.0       0.0     54582.0          8.196748              2.0                        0.0
                                                   15                                        63.0                         63.0       0.0     54582.0          8.196748              2.0                        0.0
                                       specialists 6                                          8.0                          0.0       8.0     54582.0          8.196748              2.0                        0.0
                                                   11                                        43.0                          0.0      43.0     54582.0          8.196748              2.0                        0.0
                                                   15                                       407.0                          0.0     407.0     54582.0          8.196748              2.0                        0.0
Ростовская область, Азов               graduates   15                                       280.0                        280.0       0.0     80757.0         54.774028             12.0                        0.0
                                       specialists 6                                          5.0                          0.0       5.0     80757.0         54.774028             12.0                        0.0
                                                   11                                       105.0                          0.0     105.0     80757.0         54.774028             12.0                        0.0
                                                   15                                       551.0                          0.0     551.0     80757.0         54.774028             12.0                        0.0
Ростовская область, Аксай              graduates   11                                        26.0                         26.0       0.0     48714.0         39.813440              7.0                        0.0
                                                   15                                        17.0                         17.0       0.0     48714.0         39.813440              7.0                        0.0
                                       specialists 11                                        15.0                          0.0      15.0     48714.0         39.813440              7.0                        0.0
                                                   15                                       117.0                          0.0     117.0     48714.0         39.813440              7.0                        0.0
Ростовская область, Батайск            graduates   11                                        80.0                         80.0       0.0    125522.0         42.983737              6.0                        0.0
                                                   15                                        30.0                         30.0       0.0    125522.0         42.983737              6.0                        0.0
                                       specialists 6                                          1.0                          0.0       1.0    125522.0         42.983737              6.0                        0.0
                                                   11                                        41.0                          0.0      41.0    125522.0         42.983737              6.0                        0.0
                                                   15     

In [21]:
grouped_grads["working_population"] = (
    (grouped_grads["population"] * 0.65).round(0).astype(int)
)

grouped_grads.drop(columns=["population"], inplace=True)

grouped_grads = grouped_grads.drop(
    columns=[
        "city_capacity_grads_and_cv_sum",
        "num_in_migration",
        "working_population",
        "factories_total",
    ]
).rename(columns={"graduates_per_year_forecast": "grads"})

In [22]:
"""
Получается что миграции которые мы растим влияют только на кол-во резюме
Условимся что изначально у нас все находится в равновесии
Как только мы меняем что-то -- у нас кол-во резюме умножается на процент изменения входящей миграции (было 100 - стало 200 -- умножаем на 2)
"""

'\nПолучается что миграции которые мы растим влияют только на кол-во резюме\nУсловимся что изначально у нас все находится в равновесии\nКак только мы меняем что-то -- у нас кол-во резюме умножается на процент изменения входящей миграции (было 100 - стало 200 -- умножаем на 2)\n'

In [23]:
grouped_grads.loc[
    grouped_grads["competitors_factories_num"] == 0, "competitors_factories_num"
] = 1

In [24]:
grouped_grads["grads"] = (
    grouped_grads["grads"] / grouped_grads["competitors_factories_num"]
)
grouped_grads["cv_count"] = (
    grouped_grads["cv_count"] / grouped_grads["competitors_factories_num"]
)

In [25]:
grouped_grads.reset_index(inplace=True)

In [26]:
uinput_spec_num

{'Машинист': 200, 'Оператор, аппаратчик': 300}

In [27]:
uinput_spec_num_2

{'Машинист': [0, 11, 12],
 'Оператор, аппаратчик': [0, 3, 6, 11, 12, 15, 16, 20]}

In [28]:
plant_attr_coef

0.1

In [29]:
grouped_grads

,cluster_center,type,edu_group_code,grads,cv_count,competitors_factories_num
0,"Краснодарский край, Тихорецк",graduates,11,88.0,0.0,1.0
1,"Краснодарский край, Тихорецк",graduates,15,63.0,0.0,1.0
2,"Краснодарский край, Тихорецк",specialists,6,0.0,8.0,1.0
3,"Краснодарский край, Тихорецк",specialists,11,0.0,43.0,1.0
4,"Краснодарский край, Тихорецк",specialists,15,0.0,407.0,1.0
5,"Ростовская область, Азов",graduates,15,280.0,0.0,1.0
6,"Ростовская область, Азов",specialists,6,0.0,5.0,1.0
7,"Ростовская область, Азов",specialists,11,0.0,105.0,1.0
8,"Ростовская область, Азов",specialists,15,0.0,551.0,1.0
9,"Ростовская область, Аксай",graduates,11,26.0,0.0,1.0


In [30]:
# Шаг 1: Создание нового столбца 'speciality' на основе словаря
def map_specialty(edu_code):
    for specialty, codes in uinput_spec_num_2.items():
        if edu_code in codes:
            return specialty
    return 'Другое'  # Если не нашли специальность

grouped_grads['speciality'] = grouped_grads['edu_group_code'].apply(map_specialty)

# Шаг 2: Группировка по cluster_center и speciality для подсчета выпускников и специалистов
result = grouped_grads.groupby(['cluster_center', 'speciality']).agg({
    'grads': 'sum',        # Сумма выпускников
    'cv_count': 'sum'      # Сумма специалистов
}).reset_index()

# Шаг 3: Переименование столбцов для ясности
result.rename(columns={'grads': 'total_graduates', 'cv_count': 'total_specialists'}, inplace=True)


In [31]:
uinput_spec_num

{'Машинист': 200, 'Оператор, аппаратчик': 300}

In [45]:
# Инициализируем колонки числовыми значениями
result['prov_graduates'] = pd.NA  # Используем Pandas NA
result['prov_specialists'] = pd.NA

# Проверяем ключи и значения
# print("Словарь uinput_spec_num:")
# print(uinput_spec_num)

# print("\nУникальные специальности:")
# print(result['speciality'].unique())

# Выполняем расчеты
for spec in result['speciality'].unique():
    if spec in uinput_spec_num:
        factor = uinput_spec_num[spec]
        # print(f"\nОбработка специальности '{spec}' с коэффициентом {factor}")
        
        # Логика расчета
        grads_values = result.loc[result['speciality'] == spec, 'total_graduates'].fillna(0) / factor
        specialists_values = result.loc[result['speciality'] == spec, 'total_specialists'].fillna(0) / factor
        
        # Устанавливаем значения напрямую
        result.loc[result['speciality'] == spec, 'prov_graduates'] = round(grads_values,2)
        result.loc[result['speciality'] == spec, 'prov_specialists'] = round(specialists_values,2)
    else:
        print(f"Специальность '{spec}' не найдена в словаре.")

# Проверяем итоговый результат
# print("\nИтоговый DataFrame:")
# print(result[['speciality', 'total_graduates', 'total_specialists', 'prov_graduates', 'prov_specialists']])

In [47]:
import numpy as np

# Проверяем и обновляем значения в 'prov_graduates'
result['prov_graduates'] = np.where(result['prov_graduates'] > 1, 1, result['prov_graduates'])

# Проверяем и обновляем значения в 'prov_specialists'
result['prov_specialists'] = np.where(result['prov_specialists'] > 1, 1, result['prov_specialists'])

In [48]:
result

,cluster_center,speciality,total_graduates,total_specialists,prov_graduates,prov_specialists
0,"Краснодарский край, Тихорецк",Машинист,88.0,43.0,0.44,0.22
1,"Краснодарский край, Тихорецк","Оператор, аппаратчик",63.0,415.0,0.21,1
2,"Ростовская область, Азов",Машинист,0.0,105.0,0.0,0.52
3,"Ростовская область, Азов","Оператор, аппаратчик",280.0,556.0,0.93,1
4,"Ростовская область, Аксай",Машинист,26.0,15.0,0.13,0.08
5,"Ростовская область, Аксай","Оператор, аппаратчик",17.0,117.0,0.06,0.39
6,"Ростовская область, Батайск",Машинист,80.0,41.0,0.4,0.2
7,"Ростовская область, Батайск","Оператор, аппаратчик",30.0,220.0,0.1,0.73
8,"Ростовская область, Белая Калитва",Машинист,0.0,47.0,0.0,0.24
9,"Ростовская область, Белая Калитва","Оператор, аппаратчик",68.0,164.0,0.23,0.55


In [324]:
"""
(!) Сейчас есть всё кроме непосредственной оценки (!)
А ну и задаваемое кол-во специалистов пока не учитывается
Но это следствие из отсутствия реализации расчета оценки

Эту таблицу практически в таком виде и планирую отдавать

Из того что обсуждали с СА:
условно выпускников взвесить по кол-ву предприятий тк их число ту мач большое
"""

'\n(!) Сейчас есть всё кроме непосредственной оценки (!)\nА ну и задаваемое кол-во специалистов пока не учитывается\nНо это следствие из отсутствия реализации расчета оценки\n\nЭту таблицу практически в таком виде и планирую отдавать\n\nИз того что обсуждали с СА:\nусловно выпускников взвесить по кол-ву предприятий тк их число ту мач большое\n'

In [325]:
# grouped_grads

grads  cv_count  competitors_factories_num
cluster_center                         type        edu_group_code                                             
Краснодарский край, Тихорецк           graduates   11                88.0       0.0                        1.0
                                                   15                63.0       0.0                        1.0
                                       specialists 6                  0.0       8.0                        1.0
                                                   11                 0.0      43.0                        1.0
                                                   15                 0.0     407.0                        1.0
Ростовская область, Азов               graduates   15               280.0       0.0                        1.0
                                       specialists 6                  0.0       5.0                        1.0
                                                   11                 0.0     105.0                        1.0
                                                   15                 0.0     551.0                        1.0
Ростовская область, Аксай              graduates   11                26.0       0.0                        1.0
                                                   15                17.0       0.0                        1.0
                                       specialists 11                 0.0      15.0                        1.0
                                                   15                 0.0     117.0                        1.0
Ростовская область, Батайск            graduates   11                80.0       0.0                        1.0
                                                   15                30.0       0.0                        1.0
                                       specialists 6                  0.0       1.0                        1.0
                                                   11                 0.0      41.0                        1.0
                                                   15                 0.0     219.0                        1.0
Ростовская область, Белая Калитва      graduates   15                68.0       0.0                        1.0
                                       specialists 6                  0.0       4.0                        1.0
                                                   11                 0.0      47.0                        1.0
                                                   15                 0.0     160.0                        1.0
Ростовская область, Зверево            graduates   15                18.0       0.0                        1.0
                                       specialists 6                  0.0       3.0                        1.0
                                                   11                 0.0      12.0                        1.0
                                                   15                 0.0      41.0                        1.0
Ростовская область, Каменск-Шахтинский graduates   15               235.0       0.0                        1.0
                                       specialists 6                  0.0       6.0                        1.0
                                                   11                 0.0      41.0                        1.0
                                                   15                 0.0     194.0                        1.0
Ростовская область, Новочеркасск       graduates   11               765.0       0.0                        1.0
                                                   15              2024.0       0.0                        1.0
                                       specialists 6                  0.0      10.0                        1.0
                                                   11                 0.0     132.0                        1.0
                                                   15                 0.

In [326]:
""" ------------------------- FINISH HERE -------------------------------- """

' ------------------------- FINISH HERE -------------------------------- '